# Multiple classifiers

We will attempt to create methods to train and evaluate multiple multi-output classifier and rank them based on F2 score

Given we were not able to create a good balanced dataset, we will test using original dataset with reduced features 

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [6]:
X_train=pd.read_csv('X_train.csv')
y_train=pd.read_csv('y_train.csv')
X_test=pd.read_csv('X_test.csv')
y_test=pd.read_csv('y_test.csv')
y_train_omf = y_train['Machine failure']
y_test_omf = y_test['Machine failure']
X_train_reduced = X_train.drop(columns=['Torque [Nm]', 'Process temperature [K]', 'Air temperature [K]'])
X_test_reduced = X_test.drop(columns=['Torque [Nm]', 'Process temperature [K]', 'Air temperature [K]'])


In [7]:
from sklearn.metrics import classification_report, fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

def analyze_model_performance(clf, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    y_train_omf = y_train['Machine failure']
    y_test_omf = y_test['Machine failure']
    clf.fit(X_train, y_train_omf)
    y_train_pred_omf = clf.predict(X_train)
    y_test_pred_omf = clf.predict(X_test)
    f2_train=fbeta_score(y_train_omf, y_train_pred_omf, beta=2, average='macro')
    f2_test=fbeta_score(y_test_omf, y_test_pred_omf, beta=2, average='macro')
    train_recall=recall_score(y_train_omf, y_train_pred_omf, average='macro')
    test_recall=recall_score(y_test_omf, y_test_pred_omf, average='macro')
    train_precision=precision_score(y_train_omf, y_train_pred_omf, average='macro', zero_division=0)
    test_precision=precision_score(y_test_omf, y_test_pred_omf, average='macro', zero_division=0)
    train_accuracy=accuracy_score(y_train_omf, y_train_pred_omf)
    test_accuracy=accuracy_score(y_test_omf, y_test_pred_omf)
    predictor_name = clf.__class__.__name__


    plt.figure(figsize=(15, 5))
    cm = confusion_matrix(y_train_omf, y_train_pred_omf)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Failure', 'Failure'])
    disp.plot(cmap=plt.cm.Blues, ax=plt.subplot(1, 2, 1))
    plt.title(f'{predictor_name} on Train Data')
    cm = confusion_matrix(y_test_omf, y_test_pred_omf)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Failure', 'Failure'])
    disp.plot(cmap=plt.cm.Blues, ax=plt.subplot(1, 2, 2))
    plt.title(f'{predictor_name} on Test Data')
    plt.show()


    print(" For `Machine failure` label:")
    print(predictor_name, " :")
    print(f"- F2 Score on Training Data: {f2_train}")
    print(f"- F2 Score on Test Data: {f2_test}")
    print(f"- Accuracy on Training Data: {train_accuracy}")
    print(f"- Accuracy on Test Data: {test_accuracy}")
    print(f"- Recall on Training Data: {train_recall}")
    print(f"- Recall on Test Data: {test_recall}")
    print(f"- Precision on Training Data: {train_precision}")
    print(f"- Precision on Test Data: {test_precision}")


# Multi-label / output Classifier

Given our data is actually multilabel, it will be good to produce a model that can also identify the type of failures not just `Machine failure`

However, we won't be able to use balanced dataset unless we could figure out how to create balanced data set for multi label data

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
cdt = DecisionTreeClassifier(random_state=42, max_depth=5, min_samples_split=10, min_samples_leaf=5)
cdt = MultiOutputClassifier(cdt)
cdt.fit(X_train_reduced, y_train)
y_pred=cdt.predict(X_test_reduced)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)
f2_test=fbeta_score(y_test['Machine failure'], y_pred['Machine failure'], beta=2, average='macro')
accuracy=accuracy_score(y_test['Machine failure'], y_pred['Machine failure'])
print("F2 value on test data", f2_test)
print("Accuracy on test data", accuracy)


F2 value on test data 0.9361787141519724
Accuracy on test data 0.9935


Looking at the output for 'Machine Learning' our multi-output classifier actually performed better than all other classifier that we trained previously

For machine failure:
- False Negative is very low
- False Positive increased but by a little bit
- F2 is good 0.9361787141519724
- Accuracy is 0.9935

# Conclusion

We learned few things while evaluating DecisionTreeClassifier

- Reduced Features improve the F2 and Accuracy as we have simpler model and less noises
- Multi-output classifier work well
- SMOTE Balanced dataset may not work well for our dataset given that we may accidentally introduce noises by injecting synthetic observations that may destroy the pattern of the minority class (ref: https://towardsdatascience.com/why-smote-is-not-necessarily-the-answer-to-your-imbalanced-dataset-ef19881da57a/)

